In [1]:
from utils import optimizer_utils as ou

In [2]:

system_prompt = "you are a senior python developer."

input_path = "./resources/original_script/"
output_path = "./resources/optimized_script/"

extension = ".py"

create_unitary_tests = True

# "llama3.2" or "gpt-4o-mini"
model = "gpt-4o-mini"
output_file_name = "optimized_gpt_unit_tests.py"

# call optimizer
ou.optimize_script(input_path, extension, output_path, model, system_prompt, output_file_name, create_unitary_tests)


Processing file: ./resources/original_script/original_notebook.py
▶️ Using gpt-4o-mini via OpenAI API...
📝 Notes from LLM:
 NOTE: 
In the refactored code, I made several important modifications to improve clarity, functionality, and maintainability:

1. **Renamed Function**: The function is renamed to `process_stock_data` for clarity.
2. **Docstrings**: Added comprehensive docstrings that explain the function purpose, parameters, return values, and exceptions.
3. **Error Handling**: Added checks for necessary columns in the DataFrame and included try/except blocks to catch errors elegantly during processing.
4. **Code Simplification**: Removed repetitive code and ensured that logic is clearly sectioned and separated.
5. **Unit Tests**: Added basic unit tests to check functionality and validate behavior under normal and erroneous conditions.
6. **PEP 8 Compliance**: Ensured the code adheres to PEP 8 styling guidelines.
7. **Optimized Logic**: Utilized vectorized operations and maintaine

In [3]:
# script path
script_path = output_path + output_file_name

# run the optimized script to ensure that the code runs OK
ou.run_python_script(script_path)

✅ Script executed successfully.

